In [4]:
from batter import MABFESystem
from batter.input_process import get_configure_from_file, parse_input_file

In [5]:
protein_file = 'data/7T2G_mp/protein_input.pdb'
system_file = 'data/7T2G_mp/system_input.pdb'
ligand_files = ['data/7T2G_mp/ligand_MP.pdb',
                'data/7T2G_mp/ligand_MP2.pdb',
                'data/7T2G_mp/ligand_DAMGO.pdb']
#system_inpcrd = 'data/7T2G_mp/system_input.inpcrd'
equilibrated_rst = 'data/7T2G_mp/equilibrated.rst'

input_file = 'data/input_files/abfe.in'

In [6]:
output_folder = 'test/abfe'
system = MABFESystem(folder=output_folder)

INFO | Loading an existing system: /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/abfe/
INFO | The folder does not contain fe: /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/abfe/


In [7]:
# check available input of create_system
system.create_system?

Signature:
system.create_system(
    system_name: str,
    protein_input: str,
    system_topology: str,
    ligand_paths: List[str],
    receptor_segment: str = None,
    system_coordinate: str = None,
    protein_align: str = 'name CA and resid 60 to 250',
    retain_lig_prot: bool = True,
    ligand_ph: float = 7.4,
    ligand_ff: str = 'gaff2',
    lipid_mol: List[str] = [],
    lipid_ff: str = 'lipid21',
    overwrite: bool = False,
)
Docstring:
Create a new single-ligand single-receptor system.

Parameters
----------
protein_input : str
    Path to the protein file in PDB format.
    It should be exported from Maestro,
    which means the protonation states of the protein are assigned.
    Water and ligand can be present in the file,
    but they will be removed during preparation.
system_topology : str
    PDB file of a prepared simulation system with `dabble`.
    The ligand does not need to be present.
system_coordinate : str
    The coordinate file for the system.
    The coo

In [ ]:
system.create_system(
            system_name='7T2G',
            protein_input=protein_file,
            system_topology=system_file,
            system_coordinate=equilibrated_rst,
            ligand_paths=ligand_files,
#            overwrite=True,
            lipid_mol=['POPC'])

In [ ]:
import MDAnalysis as mda

u_prot = mda.Universe(f'{output_folder}/all-poses/reference.pdb')
u_lig = mda.Universe(f'{output_folder}/all-poses/pose0.pdb')
u_merge = mda.Merge(u_prot.atoms, u_lig.atoms)

P1_atom = u_merge.select_atoms('name CA and resid 149')
P2_atom = u_merge.select_atoms('name CA and resid 119')
P3_atom = u_merge.select_atoms('name CA and resid 328')
if P1_atom.n_atoms != 1 or P2_atom.n_atoms != 1 or P3_atom.n_atoms != 1:
    raise ValueError('Error: more than one atom selected')

lig_atom = u_merge.select_atoms(f'resname {u_lig.atoms.resnames[0]} and name C12')

# get ll_x,y,z distances
r_vect = lig_atom.center_of_mass() - P1_atom.positions
print(f'l1_x: {r_vect[0][0]:.2f}')
print(f'l1_y: {r_vect[0][1]:.2f}')
print(f'l1_z: {r_vect[0][2]:.2f}')

Modify the `test/abfe/abfe.in` file to include the following lines based on the output of the above script:
    
```bash
    l1_x: xxx
    l1_y: xxx
    l1_z: xxx
```

In [ ]:
# Prepare the system for the equilibration stage
system.prepare(
    stage='equil',
    input_file=input_file,
    overwrite=True,
)

In [ ]:
# Submit the equilibration job to SLURM
system.submit(stage='equil')

Wait for the equilbration to finish.

In [ ]:
# Prepare the system for the free energy calculation
system.prepare(
    stage='fe',
    input_file=input_file,
#    overwrite=True,
)

In [ ]:
# Submit the equilibration job to SLURM
system.submit(stage='fe')

Wait for the production to finish.

In [ ]:
system.analysis()